In [82]:
import cv2
import numpy as np

class Contour:
    def __init__(self, coordinates, color):
        self.coordinates = coordinates
        self.color = color

# Define a function that returns the x, y coordinates of the contour
def get_contour_center(contour):
    contour_np = np.array(contour)
    rect = cv2.minAreaRect(contour_np)
    center = rect[0]
    return center

# Load the image
img = cv2.imread("IMG_4745.jpg")

# Convert the image to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define the range of color you want to detect

color_lower = (20, 100, 100)
color_upper = (40, 255, 255)

colors = [('yellow', (20, 100, 100), (40, 255, 255)),
    ('red', (0, 100, 100), (10, 255, 255)),
    ('blue', (100, 50, 50), (120, 255, 255)),
    ('green', (50, 50, 50), (70, 255, 255)),
    ('orange', (5, 50, 50), (15, 255, 255)),
    ('white', (20, 10, 130), (40, 50, 255))
    ]

color_map = {"red": (0, 0, 255), "green": (0, 255, 0), "blue": (255, 0, 0), "yellow": (0, 255, 255), "white": (255,255,255), "orange": (0, 165, 255)}

contoursList = []

for color, color_lower, color_upper in colors:
    # Create a binary mask where the color is within the range
    mask = cv2.inRange(hsv, color_lower, color_upper)

    # Perform morphological operations and draw squares onto the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours on the mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for i in range(len(contours)):
        cnt = contours[i]
        # Approximate the contour with a polygon
        epsilon = 0.02*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        
        area = cv2.contourArea(cnt)
        rect = cv2.minAreaRect(approx)
        angle = rect[-1]
        # check if the approximated contour has 4 sides
        if len(approx) == 4 and area > 50000 and (abs(angle) < 10 or (abs(angle) > 80 and abs(angle) < 100)) and hierarchy[0][i][3] == -1 :
            line_lengths = [cv2.norm(approx[i]-approx[(i+1)%4]) for i in range(4)]
            if all(abs(line_lengths[i]-line_lengths[i+1]) < 50 for i in range(3)):
                # Draw the contour on the original image
                contoursList.append(Contour(cnt, color))

# Sort the contours_list
contoursList = sorted(contoursList, key=lambda c: (cv2.boundingRect(c.coordinates)[1], cv2.boundingRect(c.coordinates)[0]))
if (len(contoursList) >= 9):
    for a in range(3):
        for i in range(3*a + 1,3*a + 3):
            key = contoursList[i]

            j = i-1
            while j>=0 and cv2.boundingRect(key.coordinates)[0] < cv2.boundingRect(contoursList[j].coordinates)[0]:
                contoursList[j+1] = contoursList[j]
                j -= 1
            contoursList[j+1] = key

counter = 0
for contour in contoursList:
    counter += 1
    color = color_map[contour.color]
    # Draw the contour on the image
    cv2.drawContours(img,[contour.coordinates], 0, (255,0,0), 10)
    cX = contour.coordinates[0][0][0]
    cY = contour.coordinates[0][0][1]
    cv2.putText(img, '#' + str(counter), (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 8)
    print (contour.color, "piece #", counter, "and the x coordinate is ", cv2.boundingRect(contour.coordinates)[0], " and the y coordinate is ", cv2.boundingRect(contour.coordinates)[1])

#cv2.drawContours(img, [contoursList[3].coordinates], 0, (255,0,0), 10)


# Display the original image with the squares highlighted
cv2.imshow("Squares", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(33)

green piece # 1 and the x coordinate is  476  and the y coordinate is  1212
blue piece # 2 and the x coordinate is  1206  and the y coordinate is  1260
blue piece # 3 and the x coordinate is  1921  and the y coordinate is  1259
green piece # 4 and the x coordinate is  502  and the y coordinate is  1952
blue piece # 5 and the x coordinate is  1237  and the y coordinate is  1957
green piece # 6 and the x coordinate is  1915  and the y coordinate is  1964
red piece # 7 and the x coordinate is  517  and the y coordinate is  2671
red piece # 8 and the x coordinate is  1244  and the y coordinate is  2681
green piece # 9 and the x coordinate is  1932  and the y coordinate is  2677


-1